# Export and convert saved model to tflite model TF2


## Workspace preparation

In [1]:
from google.colab import drive
drive.mount('/content/drive')
!rm -rf /content/sample_data

%cd /content/
! git clone --depth 1 https://github.com/tensorflow/models
! pip install -q tf_slim
! pip install -qq pycocotools


%cd /content/models/research/
! protoc object_detection/protos/*.proto --python_out=.
! pip install .

import tensorflow as tf
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/'


%cd /content/drive/My Drive/projects/ML/object_detection_tf/workspace

Mounted at /content/drive
/content
Cloning into 'models'...
remote: Enumerating objects: 2249, done.
remote: Counting objects: 100% (2249/2249), done.
remote: Compressing objects: 100% (1945/1945), done.
remote: Total 2249 (delta 552), reused 945 (delta 279), pack-reused 0
Receiving objects: 100% (2249/2249), 30.51 MiB | 20.12 MiB/s, done.
Resolving deltas: 100% (552/552), done.
     |████████████████████████████████| 358kB 9.0MB/s 
/content/models/research
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1350338 sha256=cd5ca6191c1c38d40bd9b856fc4761e15c304d2030d70972415efa3ff9c00215
  Stored in directory: /tmp/pip-ephem-wheel-cache-p98j7frg/wheels/94/49/4b/39b051683087a22ef7e80ec52152a27249d1a644ccf4e442ea
Successfully built object-detection
/content/drive/My Drive/projects/ML/object_detection_tf/workspace


## Exporting model

### Model selection

In [2]:
selected_model = "faster_rcnn_resnet101_v1_800x1333" #@param ["ssd_mobilenet_v2_320x320", "ssd_mobilenet_v2_fpnlite_320x320", "ssd_mobilenet_v2_fpnlite_640x640", "faster_rcnn_resnet101_v1_800x1333"]
pre_trained_model_dir = f'pre_trained_models/{selected_model}_coco17_gpu-8'
pipeline_path = f'my_models/pipelines/{selected_model}.config'
output_model_dir = f'my_models/{selected_model}'
trained_checkpoint_prefix = f'my_models/{selected_model}/model.ckpt-5'
exported_models_dir = f'exported_models/{selected_model}/'

### Inference graph

In [3]:
!python3 /content/models/research/object_detection/exporter_main_v2.py \
    --input_type=image_tensor \
    --pipeline_config_path {pipeline_path} \
    --trained_checkpoint_dir {output_model_dir} \
    --output_directory {exported_models_dir} \

2020-10-17 00:23:40.248321: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-17 00:23:43.612133: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-17 00:23:43.672362: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-17 00:23:43.672930: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-10-17 00:23:43.672970: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-17 00:23:43.952833: I tensorflow/stream_executor/platform/default

### TFlite SSD graph

In [ ]:
!python3 /content/models/research/object_detection/export_tflite_graph_tf2.py \
    --pipeline_config_path {pipeline_path} \
    --trained_checkpoint_dir {output_model_dir} \
    --output_directory {exported_models_dir}

## Converting model to TF model

In [ ]:
export_dir = f'{exported_models_dir}saved_model'
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir,signature_keys=['serving_default'])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter = True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

In [ ]:
# Save the TF Lite model.
with tf.io.gfile.GFile(f'{exported_models_dir}detect.tflite', 'wb') as f:
  f.write(tflite_model)